<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Ejem_Peluqueria_Linea_de_espera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from random import random, seed
import pandas as pd
import simpy

In [ ]:
def peluqueria(env, pelucas, nc, texto=True):
    global  ll, cifras, dur_corte, t_esp, fin_sim, LTLL, LC, LS

    for i in range(nc):
        #Llegada del cliente
        r = random()
        tll = -np.log(r)/ll
        yield env.timeout(tll)
        LTLL.append(env.now)
        if texto:
          print(f'----Ha llegado el cliente número {i} en el minuto {round(env.now, cifras)}----')

        env.process(corte(env, pelucas, i, texto))

In [ ]:
def corte(env, pelucas, num, texto=True):
    global tc_min, tc_max, cifras, dur_corte, LC, LS
    a, b = tc_min, tc_max
    #Solicitud del corte
    with pelucas.request() as sol:
        yield sol

        #Inicia el corte

        #Se genera el tiempo de corte
        r = random()
        tc = a + (b-a)*r
        dur_corte += tc
        LC.append(tc)
        if texto:
          print(f'****Inicia el corte del cliente {num} en el minuto {round(env.now, cifras)}****')
        #Se suma el tiempo a la line ade tiempo actual
        yield env.timeout(tc)

        #Muestra un mensaje al término del corte
        LS.append(env.now)
        if texto:
          print(f'xxxTermina el corte del cliente {num} en el minuto {round(env.now, cifras)}xxx')
        return tc

In [ ]:
global tc_min, tc_max, ll, no_peluqueros, cifras
semilla = 13
seed(semilla)

no_peluqueros = 1
tc_min = 15
tc_max = 30
ll = 1/20
clientes = 5
cifras = 2

In [ ]:
global t_esp, dur_corte, fin_sim, LTLL, LC, LS
t_esp = 0
dur_corte = 0
fin_sim = 0

LTLL, LC, LS = [], [], []

In [ ]:
ent = simpy.Environment()
pelucas = simpy.Resource(ent, capacity=no_peluqueros)

In [ ]:
ent.process(peluqueria(ent, pelucas, clientes))

In [ ]:
ent.run()

In [ ]:
LTLL, LC, LS = np.array(LTLL), np.array(LC), np.array(LS)
t_esp = sum(LS - LTLL - LC)
fin_sim = ent.now

In [ ]:
print(LTLL)
print(LC)
print(LS)
print(t_esp, dur_corte, fin_sim)

In [ ]:
print(f'Longitud promedio de la cola (fila): {round(t_esp/fin_sim, 2)} personas en cola')
print(f'Tiempo de espera promedio: {round(t_esp/clientes,2)} minutos')
print(f'Uso promedio de la instalación: {round(100 * dur_corte/fin_sim,2)}%')

In [ ]:
def exp(n):
  global tc_min, tc_max, ll, no_peluqueros, t_esp, dur_corte, fin_sim, LTLL, LC, LS

  no_peluqueros = 1
  tc_min = 15
  tc_max = 30
  ll = 1/20
  clientes = 5
  cifras = 2

  long_cola = []
  tiempo_esp = []
  uso = []

  for _ in range(n):
    t_esp = 0
    dur_corte = 0
    fin_sim = 0

    LTLL, LC, LS = [], [], []

    ent = simpy.Environment()
    pelucas = simpy.Resource(ent, capacity=no_peluqueros)
    ent.process(peluqueria(ent, pelucas, clientes, texto=False))
    ent.run()
    LTLL, LC, LS = np.array(LTLL), np.array(LC), np.array(LS)
    t_esp = sum(LS - LTLL - LC)
    fin_sim = ent.now

    long_cola.append(t_esp/fin_sim)
    tiempo_esp.append(t_esp/clientes)
    uso.append(100 * dur_corte/fin_sim)

  return long_cola, tiempo_esp, uso

In [ ]:
n = 500
a, b, c = exp(n)

In [ ]:
def graf_estado(l, titulo):
  g = []
  n = len(l)
  for i in range(n):
    g.append(np.mean(l[:i+1]))
  plt.plot(g, 'lightblue')
  m = np.mean(l)
  s = np.std(l, ddof=1)
  plt.hlines(m, 0, n, color='r', linestyles='dotted')
  plt.hlines(m + 1.96*np.std(l)/np.sqrt(n), 0, n, color='g', linestyles='dashed')
  plt.hlines(m - 1.96*np.std(l)/np.sqrt(n), 0, n, color='g', linestyles='dashed')
  plt.title(titulo)
  plt.xlabel('iteraciones')
  plt.show()

In [ ]:
plt.style.use('dark_background')
graf_estado(a, 'Gráfica de la longitud promedio de la cola (fila)')
graf_estado(b, 'Gráfica del tiempo de espera promedio')
graf_estado(c, 'Gráfica del uso promedio de la instalación')

In [ ]:
def rep(n, nr):
  la, lb, lc = [], [], []
  for _ in range(nr):
    a, b, c = exp(n)
    la.append(np.mean(a))
    lb.append(np.mean(b))
    lc.append(np.mean(c))
  return la, lb, lc

In [ ]:
n = 500
nr = 100
a, b, c = rep(n, nr)

In [ ]:
from scipy.stats import t
ma, mb, mc = np.mean(a), np.mean(b), np.mean(c)
sa, sb, sc = np.std(a, ddof=1), np.std(b, ddof=1), np.std(c, ddof=1)
t_student = t.ppf(0.975, nr-1)
# print(t_student)
intervalo_a = (ma - t_student*sa/np.sqrt(nr), ma + t_student*sa/np.sqrt(nr))
intervalo_b = (mb - t_student*sb/np.sqrt(nr), mb + t_student*sb/np.sqrt(nr))
intervalo_c = (mc - t_student*sc/np.sqrt(nr), mc + t_student*sc/np.sqrt(nr))
dec = 5

In [ ]:
print(f'El intervalo de confianza al 95% de la longitud promedio de la cola (fila): {np.round(intervalo_a, dec)} personas en cola')
print(f'El intervalo de confianza al 95% del tiempo de espera promedio: {np.round(intervalo_b, dec)} minutos')
print(f'El intervalo de confianza al 95% del uso promedio de la instalación: {np.round(intervalo_c, dec)}%')